<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Project 3: NLP Analysis of Reddit Data 
## Part 2: Data Cleaning

Katrin Ayrapetov 

In [2]:
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import re


In [3]:
comments_elem_df =  pd.read_csv('comments_elem_df.csv')

In [7]:
submissions_elem_df =  pd.read_csv('submissions_elem_df.csv')

In [11]:
submissions_teach_ms =  pd.read_csv('submissions_teach_ms.csv')

In [14]:
df_hs =  pd.read_csv('df_hs.csv')

In [15]:
#Join the comments and submissions data sets 
df_elem = pd.concat([comments_elem_df, submissions_elem_df], axis=0)


In [17]:
#Remove rows with comments that have been deleted by the moderator or removed by the user. 
df_elem.drop(df_elem[df_elem['body'] == "[deleted]"].index, inplace = True)
df_elem.drop(df_elem[df_elem['body'] == "[removed]"].index, inplace = True)
df_hs.drop(df_hs[df_hs['body'] == "[removed]"].index, inplace = True)
df_hs.drop(df_hs[df_hs['body'] == "[deleted]"].index, inplace = True)



In [18]:
#Reset the index 
df_elem.reset_index(drop="True",inplace=True)
df_hs.reset_index(drop="True",inplace=True)

In [19]:
#Remove duplicates and keep the first 
df_elem.drop_duplicates(keep = "first", inplace = True)
df_hs.drop_duplicates(keep = "first", inplace = True)
#Reset the index after dropping the duplicates 
df_elem.reset_index(drop="True",inplace=True)
df_hs.reset_index(drop="True",inplace=True)

In [20]:
#Overwrrite the string with any internet address that begins in "http" or ends with ".com" removed.
#for the elementary school data set. 

for i in range(df_elem.shape[0]):
    re.sub(r'http\S+', '', df_elem.iloc[i]['body'].lower())
    re.sub(r'\S+.com', '', df_elem.iloc[i]['body'].lower())
    


In [23]:
#Overwrrite the string with any internet address that begins in "http" or ends with ".com" removed
#for the high school data set. 

for i in range(df_hs.shape[0]):
    re.sub(r'http\S+', '', df_hs.iloc[i]['body'].lower())
    re.sub(r'\S+.com', '', df_hs.iloc[i]['body'].lower())
    

In [24]:
# Get rid of accent marks, umlats or any other above letter symbols.  
import unicodedata
df_elem['body'] = df_elem['body'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_hs['body'] = df_hs['body'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [25]:
### Expand contracted words in the elementary teachers data set, so that "can't" will read "cannot" #### 
import contractions 

for i in range(df_elem.shape[0]):
    expanded_words = []
    for word in df_elem.iloc[i]['body'].split():
        expanded_words.append(contractions.fix(word))
        df_elem.iloc[i]['body'] = ' '.join(expanded_words)

In [27]:
### Expand contracted words in the high school data set, so that "can't" will read "cannot" #### 
import contractions 

for i in range(df_hs.shape[0]):
    expanded_words = []
    for word in df_hs.iloc[i]['body'].split():
        expanded_words.append(contractions.fix(word))
        df_hs.iloc[i]['body'] = ' '.join(expanded_words)

In [28]:
#Create an empty column to store cleaned comments for elementary data set. 
df_elem['CLEAN_COMMENT'] = df_elem.apply(lambda _: '', axis=1)

In [30]:
#Create an empty column to store cleaned comments for middle school data set. 
df_hs['CLEAN_COMMENT'] = df_hs.apply(lambda _: '', axis=1)

In [31]:
#create a list called "cheat list" that includes "stop words" particular to our problem 
cheat_words = ["elementary", "Elementary","Elementary","middle","high","High","school","School"]

In [32]:
#Createa a function that takes in a dataframe, the name of the column where strings are stored, 
# and the name of the column where cleaned string will be stored. The function clean_comments does 
# the following: 
# (1) Removes special symbols and numbers
# (2) Removes stop words
# (3) Removes words in "cheat list"
# (3) Changes the words to the lemmatized version 
# (4) Creates a new clean string 


def clean_comments(df,column_name,new_column_name):
    dim = df.shape[0]
    for i in range(dim):
        # Remove the punctuation .
        # This creates a list called "words_sans_punctuation" which contains all the words in the status
        #with punctuations,numbers, and special symbols removed.
        words_sans_punctuation = re.findall(r'[a-z]*',df.iloc[i][column_name].lower())
        # This creates a list called "words_sans_stop_words" which contains all the words in the status
        #with punctuations,numbers, and special symbols removed. 
        words_sans_cheat_words = [w for w in list(filter(("").__ne__, words_sans_punctuation)) if not w in cheat_words]
        words_sans_stop_words = [w for w in list(filter(("").__ne__, words_sans_cheat_words)) if not w in stopwords.words('english')]
        
        # Replace the words with their lemmatized version. 
        # This creates a list called "lemms_of_words" which contains the lemmatized version
        # of all the words in the facebook status. 
        lemms_of_words = []
        lemmatizer = WordNetLemmatizer()
        for w in list(filter(("").__ne__, words_sans_stop_words)):
            lemms_of_words.append(lemmatizer.lemmatize(w))
        
        from spellchecker import SpellChecker
        spell = SpellChecker()
        words_spell_checked = [spell.correction(word) for word in list(filter(("").__ne__, lemms_of_words))]
        # Creates a new clean string 
        cleaned_status  = " ".join(list(filter(("").__ne__, words_spell_checked)))
        # Stores the new clean string into a new column 
 
        df.at[i, new_column_name] = cleaned_status
        

In [33]:
clean_comments(df_elem,'body','CLEAN_COMMENT')

In [35]:
clean_comments(df_hs,'body','CLEAN_COMMENT')

In [36]:
#Export the clean dataframe to Excel SpreadSheets 
df_elem.to_csv('df_elem_CLEAN.csv', header=True, index=False)


In [38]:
df_hs.to_csv('df_hs_CLEAN.csv', header=True, index=False)

In [39]:
df_elem_clean =  pd.read_csv('df_elem_CLEAN.csv')

df_hs_clean =  pd.read_csv('df_hs_CLEAN.csv')

In [62]:
#Extract only the adjectives and the adverbs 

In [41]:
df_clean_elem_hs = pd.concat([df_elem, df_hs], axis=0, ignore_index=True)

In [42]:

#Drop the null values. 
df_clean_elem_hs.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
#Reset the index 
df_clean_elem_hs.reset_index(drop="True",inplace=True)

In [45]:
df_clean_elem_hs.shape

(5519, 3)

In [46]:
#Create an empty column to store only adjectives and adverbs of the data for middle school data set. 
df_clean_elem_ms['ADJ_ADV_CLEAN_COMMENT'] = df_clean_elem_ms.apply(lambda _: '', axis=1)

In [47]:
#Create an empty column to store only adjectives and adverbs of the data for middle school data set. 
df_clean_elem_hs['ADJ_ADV_CLEAN_COMMENT'] = df_clean_elem_hs.apply(lambda _: '', axis=1)

In [48]:
def adj_adv_extractor(df,column_name,new_column_name):
    dim = df.shape[0]
    from nltk import pos_tag
    for i in range(dim):
        words_with_pos = pos_tag(df.iloc[i][column_name].split())
        list_of_parts_of_speech = ["JJ","JJR","JJS"]
        adverbs_and_adjectives = []
        for pair in words_with_pos:
            if pair[1] in list_of_parts_of_speech:
                adverbs_and_adjectives.append(pair[0])
        
            adj_clean_text  = " ".join(list(filter(("").__ne__, adverbs_and_adjectives)))
            df.at[i, new_column_name] = adj_clean_text
        

In [49]:
adj_adv_extractor(df_clean_elem_ms,'CLEAN_COMMENT','ADJ_ADV_CLEAN_COMMENT')

In [51]:

#Drop the null values. 
df_clean_elem_hs.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
#Reset the index 
df_clean_elem_hs.reset_index(drop="True",inplace=True)

In [53]:
df_clean_elem_hs.head()

,subreddit,body,CLEAN_COMMENT,ADJ_ADV_CLEAN_COMMENT
0,ElementaryTeachers,Representation is important in the classroom. ...,representation important classroom every child...,important least
1,ElementaryTeachers,anything to keep math from being too dry (and ...,anything keep math dry intimidating student ab...,keep dry much great able different different
2,ElementaryTeachers,-Sometimes redirection is just the name of the...,sometimes redirection name game honestly say r...,good general copy teacher sure table fun kid y...
3,ElementaryTeachers,"If it feels like it should not be happening, i...",feel like happening probably happening sort be...,arm sort common positive good respectful respo...
4,ElementaryTeachers,We all die soon. May as well make this life wo...,die soon may well make life worth positive eve...,positive everyday pal positive


In [54]:
df_clean_elem_hs.to_csv('df_clean_elem_hs.csv', header=True, index=False)